# Wrapping a subset of a very large library

Sometimes, for a very large library, only a subset of available *C++* components is useful for end-users.
Wrapping such libraries therefore requires **AutoWIG** to be able to consider only a subset of the *C++* components during the `Generate` step.
The **Clang** library is a complete *C*/*C++* compiler.
**Clang** is a great tool, but its stable *Python* interface (i.e. **libclang**) is lacking some useful features that are needed by **AutoWIG**.
In particular, class template specializations are not available in the abstract syntax tree.
Fortunately, most of the classes that would be needed during the traversal of the *C++* abstract syntax tree are not template specializations.
We therefore proposed to bootstrap the **Clang** *Python* bindings using the `libclang` `parser` of **AutoWIG**.
This new **Clang** *Python* interface is called **PyClangLite** and is able to parse class template specializations.
As for **libclang**, this interface is proposed only for a subset of the **Clang** library sufficient enough for proposing the new `pyclanglite` `parser`.

First, ensure that the **LLVM**/**Clang** technologies are installed on your computer and if there were built with `RTTI`.

In [1]:
import subprocess
subprocess.check_call(["llvm-config", "--version"])
subprocess.check_call(['clang++', '--version'])
if not subprocess.check_output(["llvm-config", "--has-rtti"]).strip() == 'YES':
    raise Exception('LLVM and Clang libraries have not been built with RTTI')

Note that these techonologies must be installed using the current system prefix

In [2]:
import sys
from path import path
prefix = path(sys.prefix).abspath()
if not prefix == subprocess.check_output(["llvm-config", "--prefix"]).strip():
    raise Exception('LLVM and Clang libraries have not been built with the \'' + prefix + '\' prefix')

This repository already has wrappers, we therefore need to remove them.

In [3]:
srcdir = path('ClangLite')/'src'/'py'
for wrapper in srcdir.walkfiles('*.cpp'):
    wrapper.unlink()
for wrapper in srcdir.walkfiles('*.h'):
    wrapper.unlink()
wrapper = srcdir/'clanglite'/'_clanglite.py'
if wrapper.exists():
    wrapper.unlink()

In addition to the **Clang** libraries, the **ClangLite** library is needed in order to have access to some functionalities.
The `tool.h` header of this **ClangLite** library includes all necessary **Clang** headers.
This library is installed using the **SCons** `cpp`  target.

In [4]:
!scons cpp -C ClangLite

scons: Entering directory `/home/pfernique/Desktop/develop/AutoWIG/doc/examples/ClangLite'
scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Building targets ...
scons: `cpp' is up to date.
scons: done building targets.


Once these preliminaries done, we can proceed to the actual generation of wrappers for the **Clang** library.
For this, we import **AutoWIG** and create an empty Abstract Semantic Graph (ASG).

In [5]:
import autowig
asg = autowig.AbstractSemanticGraph()

We then parse the `tool.h` header of the **ClangLite** library with relevant compilation flags.

In [6]:
%%time
autowig.parser.plugin = 'libclang'
asg = autowig.parser(asg, [prefix/'include'/'clanglite'/'tool.h'],
               flags = ['-x', 'c++', '-std=c++11',
                        '-D__STDC_LIMIT_MACROS',
                        '-D__STDC_CONSTANT_MACROS',
                        '-I' + str((prefix/'include').abspath())],
               libpath = prefix/'lib'/'libclang.so',
               bootstrap = False,
               silent = True)

CPU times: user 2min 21s, sys: 428 ms, total: 2min 21s
Wall time: 2min 22s


Since most of **AutoWIG** guidelines are respected in the **Clang** library, the `default` `controller` implementation could be suitable.
Nevertheless, we need to force some *C++* components to be wrapped or not.
We therefore implements a new `controller`.

In [7]:
def clanglite_controller(asg):
    
    for node in asg['::boost::python'].classes(nested = True):
        node.is_copyable = True
        
    for node in asg.classes():
        node.boost_python_export = False
    for node in asg.enumerations():
        node.boost_python_export = False
    for node in asg.enumerators():
        if node.parent.boost_python_export:
            node.boost_python_export = False
    for node in asg.typedefs():
        node.boost_python_export = False
            
    from autowig.default_controller import refactoring
    asg = refactoring(asg)
    for fct in asg['::clanglite'].functions():
        if not fct.localname == 'build_ast_from_code_with_args':
            fct.parent = fct.parameters[0].qualified_type.desugared_type.unqualified_type
            
    for node in asg.functions(free = True):
        node.boost_python_export = False
    for node in asg.variables(free = True):
        node.boost_python_export = False
    
    if autowig.parser.plugin == 'libclang':
        for fct in asg.functions(free=False):
            asg._nodes[fct._node]['_is_virtual'] = False
            asg._nodes[fct._node]['_is_pure'] = False
        asg['class ::clang::QualType'].is_abstract = False
        asg['class ::clang::QualType'].is_copyable = True
        asg['class ::llvm::StringRef'].is_abstract = False
        asg['class ::llvm::StringRef'].is_copyable = True
        asg['class ::clang::FileID'].is_abstract = False
        asg['class ::clang::FileID'].is_copyable = True
        asg['class ::clang::SourceLocation'].is_abstract = False
        asg['class ::clang::SourceLocation'].is_copyable = True
        asg['class ::clang::TemplateArgument'].is_abstract = False
        asg['class ::clang::TemplateArgument'].is_copyable = True
        for cls in ['::clang::FriendDecl', '::clang::CapturedDecl', '::clang::OMPThreadPrivateDecl',
                    '::clang::NonTypeTemplateParmDecl', '::clang::TemplateArgumentList', '::clang::ImportDecl',
                    '::clang::TemplateTemplateParmDecl', '::clang::CapturedDecl', '::clang::OMPThreadPrivateDecl',
                    '::clang::NonTypeTemplateParmDecl', '::clang::TemplateArgumentList', '::clang::ImportDecl',
                    '::clang::TemplateTemplateParmDecl']:
            asg['class ' + cls].is_abstract = False
        
    asg['class ::boost::python::api::object'].boost_python_export = True
    asg['class ::boost::python::list'].boost_python_export = True 
    asg['class ::boost::python::str'].boost_python_export = True 

    subset = []
    classes = [asg['class ::clang::QualType'],
               asg['class ::clang::Type'],
               asg['class ::clang::Decl']]

    subset += classes
    for cls in classes:
        subset += cls.subclasses(recursive=True)
    subset.append(asg['class ::llvm::StringRef'])

    subset.append(asg['class ::clang::ASTUnit'])
    subset.append(asg['class ::clang::ASTContext'])
    subset.append(asg['class ::clang::SourceManager'])
    subset.append(asg['class ::clang::FileID'])

    subset.append(asg['class ::clang::SourceLocation'])

    subset.append(asg['class ::clang::CXXBaseSpecifier'])
    subset.append(asg['class ::clang::DeclContext'])
    subset.append(asg['class ::clang::TemplateArgument'])

    subset.append(asg['class ::clang::TemplateArgumentList'])
    subset.append(asg['enum ::clang::Type::TypeClass'])
    subset.append(asg['enum ::clang::AccessSpecifier'])
    subset.append(asg['enum ::clang::LinkageSpecDecl::LanguageIDs'])
    subset.append(asg['enum ::clang::BuiltinType::Kind'])
    subset.append(asg['enum ::clang::TemplateArgument::ArgKind'])
    subset.append(asg['enum ::clang::Decl::Kind'])
    subset.extend(asg['::boost::python'].classes(nested = True))
    subset.extend(asg['::boost::python'].enumerations(nested = True))
    subset.extend(asg.nodes('::clanglite::build_ast_from_code_with_args'))

    for node in subset:
        node.boost_python_export = True

    for mtd in asg['class ::clang::ASTContext'].methods(pattern='.*getSourceManager.*'):
        if mtd.return_type.globalname == 'class ::clang::SourceManager &':
                mtd.boost_python_export = True
                break
                
    if autowig.parser.plugin == 'libclang':
        for node in (asg.functions(pattern='.*(llvm|clang).*_(begin|end)')
                     + asg.functions(pattern='.*(llvm|clang).*getNameAsString')
                     + asg.nodes('::clang::NamedDecl::getQualifiedNameAsString')
                     + asg.nodes('::clang::ObjCProtocolDecl::collectInheritedProtocolProperties')
                     + asg.nodes('::clang::ASTUnit::LoadFromASTFile')
                     + asg.nodes('::clang::ASTUnit::getCachedCompletionTypes')
                     + asg.nodes('::clang::ASTUnit::getBufferForFile')
                     + asg.nodes('::clang::CXXRecordDecl::getCaptureFields')
                     + asg.nodes('::clang::ASTContext::SectionInfos')
                     + asg.nodes('::clang::ASTContext::getAllocator')
                     + asg.nodes('::clang::ASTContext::getObjCEncoding.*')
                     + asg.nodes('::clang::ASTContext::getAllocator')
                     + asg.nodes('::clang::QualType::getAsString')
                     + asg.nodes('::clang::SourceLocation::printToString')
                     + asg['class ::llvm::StringRef'].methods()):
            node.boost_python_export = False
            
    import sys
    from path import path
    for header in (path(sys.prefix)/'include'/'clang').walkfiles('*.h'):
        asg[header.abspath()].is_external_dependency = False
    
    return asg

This `controller` is then dynamically registered and used on the ASG.

In [8]:
%%time
autowig.controller['clanglite'] = clanglite_controller
autowig.controller.plugin = 'clanglite'
asg = autowig.controller(asg)

CPU times: user 18.5 s, sys: 48 ms, total: 18.5 s
Wall time: 18.5 s


In order to wrap a subset of the **Clang** library, we need to select the `boost_python_internal` `generator` implementation.

In [9]:
%%time
autowig.generator.plugin = 'boost_python_pattern'
wrappers = autowig.generator(asg,
                  module = srcdir/'_clanglite.cpp',
                  decorator = srcdir/'clanglite'/'_clanglite.py',
                  closure = False)

CPU times: user 9.42 s, sys: 8 ms, total: 9.42 s
Wall time: 9.38 s


The wrappers are only generated in-memory.
It is therefore needed to write them on the disk to complete the process.

In [10]:
%%time
wrappers.write()

CPU times: user 2min 8s, sys: 40 ms, total: 2min 8s
Wall time: 2min 8s


Here is an example of the generated wrappers.
We here present the wrappers for the `clang::Decl` class.

In [11]:
!pygmentize ClangLite/src/py/wrapper_a6aedb4654a55a40aeecf4b1dc5fcc98.cpp

#include "_clanglite.h"



namespace autowig
{
    class Wrap_a6aedb4654a55a40aeecf4b1dc5fcc98 : public ::clang::Decl, public boost::python::wrapper< class ::clang::Decl >
    {
        public:
            

        protected:
            

        private:
            

    };

}

#if defined(_MSC_VER)
    #if (_MSC_VER == 1900)
namespace boost
{
    template <> autowig::Wrap_a6aedb4654a55a40aeecf4b1dc5fcc98 const volatile * get_pointer<autowig::Wrap_a6aedb4654a55a40aeecf4b1dc5fcc98 const volatile >(autowig::Wrap_a6aedb4654a55a40aeecf4b1dc5fcc98 const volatile *c) { return c; }
    template <> class ::clang::Decl const volatile * get_pointer<class ::clang::Decl const volatile >(class ::clang::Decl const volatile *c) { return c; }
}
    #endif
#endif



void wrapper_a6aedb4654a55a40aeecf4b1dc5fcc98()
{

    std::string name_7bbff48d109853e88270b3595c663a99 = boost::python::extract< std::string >(boost::python::scope().attr("__name__") + ".clang");
    boost::python::object module_7bbff

    bool  (::clang::Decl::*method_pointer_e5ef166b28485ea4aec073822fa15031)() const = &::clang::Decl::isTemplateParameterPack;
    bool  (::clang::Decl::*method_pointer_f676d859f5f25558a0d9c5ab109fbc40)() const = &::clang::Decl::isThisDeclarationReferenced;
    bool  (::clang::Decl::*method_pointer_020f3cd4e34a53f5af7e7610abb64793)() const = &::clang::Decl::isTopLevelDeclInObjCContainer;
    bool  (::clang::Decl::*method_pointer_bc71996000e95d46874831b3726bb949)(bool ) const = &::clang::Decl::isUsed;
    bool  (::clang::Decl::*method_pointer_2e1e01fc7c115b35ac4a4364d9d9da2d)() const = &::clang::Decl::isWeakImported;
    void  (::clang::Decl::*method_pointer_69d255320527548c909912927fc00914)(class ::clang::ASTContext &) = &::clang::Decl::markUsed;
    void  (::clang::Decl::*method_pointer_a62d828efcb557e6a83ef5e72f466936)(enum ::clang::AccessSpecifier ) = &::clang::Decl::setAccess;
    void  (::clang::Decl::*method_pointer_357ee2dd08135691a1d77fd9fc025e57)(class ::clang::DeclContext *) 

Once the wrappers are written on the disk, the bingings must be compiled and installed.
This can be done using the **SCons** `py`  target.

In [12]:
!conda build -q ClangLite/conda/python-clanglite -c statiskit
!conda install -y -q python-clanglite --use-local -c statiskit --force

BUILD START: python-clanglite-3.8.1-py27_0

The following NEW packages will be INSTALLED:

    clang:        3.8.1-0       local
    coverage:     4.3.4-py27_0       
    icu:          54.1-0             
    libboost:     1.61.0-py27_0 local
    libclanglite: 3.8.1-py27_0  local
    libdev:       1.0.0-py27_0  local
    librun:       1.0.0-py27_0  local
    llvm:         3.8.1-0       local
    nose:         1.3.7-py27_1       
    openssl:      1.0.2k-1           
    path.py:      10.3.1-py27_0      
    pip:          9.0.1-py27_1       
    python:       2.7.13-0           
    python-dev:   1.0.0-py27_0  local
    python-scons: 2.5.0-py27_0  local
    pyyaml:       3.12-py27_0        
    readline:     6.2-2              
    setuptools:   27.2.0-py27_0      
    sqlite:       3.13.0-0           
    tk:           8.5.18-0           
    wheel:        0.29.0-py27_0      
    yaml:         0.1.6-0            
    zlib:         1.2.8-3            


latest version is 2.1.12. Run

co

g++ -o build/py/wrapper_af4470701d7f5ca1967ae6676dc947da.os -c -x c++ -std=c++11 -fvisibility-inlines-hidden -ffunction-sections -fdata-sections -Wno-deprecated-declarations -fmax-errors=0 -Wl,--no-undefined -fvisibility=hidden -fPIC -DBOOST_PYTHON_DYNAMIC_LIB -DBOOST_ALL_NO_LIB -D_GNU_SOURCE -D__STDC_CONSTANT_MACROS -D__STDC_FORMAT_MACROS -D__STDC_LIMIT_MACROS -I/home/pfernique/.miniconda/conda-bld/python-clanglite_1494419543177/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include -I/home/pfernique/.miniconda/conda-bld/python-clanglite_1494419543177/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/python2.7 build/py/wrapper_af4470701d7f5ca1967ae6676dc947da.cpp
g++ -o build/py/wrapper_0418dea67efc501b

g++ -o build/py/wrapper_2fbc20c1c02555b8b7a912504bb82c73.os -c -x c++ -std=c++11 -fvisibility-inlines-hidden -ffunction-sections -fdata-sections -Wno-deprecated-declarations -fmax-errors=0 -Wl,--no-undefined -fvisibility=hidden -fPIC -DBOOST_PYTHON_DYNAMIC_LIB -DBOOST_ALL_NO_LIB -D_GNU_SOURCE -D__STDC_CONSTANT_MACROS -D__STDC_FORMAT_MACROS -D__STDC_LIMIT_MACROS -I/home/pfernique/.miniconda/conda-bld/python-clanglite_1494419543177/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include -I/home/pfernique/.miniconda/conda-bld/python-clanglite_1494419543177/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/python2.7 build/py/wrapper_2fbc20c1c02555b8b7a912504bb82c73.cpp
In file included from /home/pfernique/.m

g++ -o build/py/wrapper_ee4ed14ebca95be7b6de45571cde69c6.os -c -x c++ -std=c++11 -fvisibility-inlines-hidden -ffunction-sections -fdata-sections -Wno-deprecated-declarations -fmax-errors=0 -Wl,--no-undefined -fvisibility=hidden -fPIC -DBOOST_PYTHON_DYNAMIC_LIB -DBOOST_ALL_NO_LIB -D_GNU_SOURCE -D__STDC_CONSTANT_MACROS -D__STDC_FORMAT_MACROS -D__STDC_LIMIT_MACROS -I/home/pfernique/.miniconda/conda-bld/python-clanglite_1494419543177/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include -I/home/pfernique/.miniconda/conda-bld/python-clanglite_1494419543177/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/python2.7 build/py/wrapper_ee4ed14ebca95be7b6de45571cde69c6.cpp
In file included from /home/pfernique/.m

In file included from /home/pfernique/.miniconda/conda-bld/python-clanglite_1494419543177/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/boost/type_traits/is_convertible.hpp:28:0,
                 from /home/pfernique/.miniconda/conda-bld/python-clanglite_1494419543177/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/boost/type_traits/is_empty.hpp:12,
                 from /home/pfernique/.miniconda/conda-bld/python-clanglite_1494419543177/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/boost/type_traits/object_traits.hpp:27,
                 from /h

...

In file included from /home/pfernique/.miniconda/conda-bld/python-clanglite_1494419543177/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/boost/type_traits/is_convertible.hpp:28:0,
                 from /home/pfernique/.miniconda/conda-bld/python-clanglite_1494419543177/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/boost/type_traits/is_empty.hpp:12,
                 from /home/pfernique/.miniconda/conda-bld/python-clanglite_1494419543177/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/boost/type_traits/object_traits.hpp:27,
                 from /h

g++ -o build/py/wrapper_0a741a99c7fd512dbe7c31ae23782c5f.os -c -x c++ -std=c++11 -fvisibility-inlines-hidden -ffunction-sections -fdata-sections -Wno-deprecated-declarations -fmax-errors=0 -Wl,--no-undefined -fvisibility=hidden -fPIC -DBOOST_PYTHON_DYNAMIC_LIB -DBOOST_ALL_NO_LIB -D_GNU_SOURCE -D__STDC_CONSTANT_MACROS -D__STDC_FORMAT_MACROS -D__STDC_LIMIT_MACROS -I/home/pfernique/.miniconda/conda-bld/python-clanglite_1494419543177/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include -I/home/pfernique/.miniconda/conda-bld/python-clanglite_1494419543177/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/python2.7 build/py/wrapper_0a741a99c7fd512dbe7c31ae23782c5f.cpp
g++ -o build/py/wrapper_60dfe77022be5b63

In file included from /home/pfernique/.miniconda/conda-bld/python-clanglite_1494419543177/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/boost/type_traits/is_convertible.hpp:28:0,
                 from /home/pfernique/.miniconda/conda-bld/python-clanglite_1494419543177/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/boost/type_traits/is_empty.hpp:12,
                 from /home/pfernique/.miniconda/conda-bld/python-clanglite_1494419543177/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/boost/type_traits/object_traits.hpp:27,
                 from /h

g++ -o build/py/wrapper_d0a4d455ddf25a6eaff261924ecb6a85.os -c -x c++ -std=c++11 -fvisibility-inlines-hidden -ffunction-sections -fdata-sections -Wno-deprecated-declarations -fmax-errors=0 -Wl,--no-undefined -fvisibility=hidden -fPIC -DBOOST_PYTHON_DYNAMIC_LIB -DBOOST_ALL_NO_LIB -D_GNU_SOURCE -D__STDC_CONSTANT_MACROS -D__STDC_FORMAT_MACROS -D__STDC_LIMIT_MACROS -I/home/pfernique/.miniconda/conda-bld/python-clanglite_1494419543177/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include -I/home/pfernique/.miniconda/conda-bld/python-clanglite_1494419543177/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/python2.7 build/py/wrapper_d0a4d455ddf25a6eaff261924ecb6a85.cpp
g++ -o build/py/wrapper_2c0e35bd45075f9c

scons: *** [build/py/wrapper_c42043d509bb5a8e8580c7560e9f5776.os] Source `build/py/wrapper_c42043d509bb5a8e8580c7560e9f5776.cpp' not found, needed by target `build/py/wrapper_c42043d509bb5a8e8580c7560e9f5776.os'.
scons: building terminated because of errors.
Traceback (most recent call last):
  File "/home/pfernique/.miniconda/bin/conda-build", line 6, in <module>
    sys.exit(conda_build.cli.main_build.main())
  File "/home/pfernique/.miniconda/lib/python2.7/site-packages/conda_build/cli/main_build.py", line 334, in main
    execute(sys.argv[1:])
  File "/home/pfernique/.miniconda/lib/python2.7/site-packages/conda_build/cli/main_build.py", line 325, in execute
    noverify=args.no_verify)
  File "/home/pfernique/.miniconda/lib/python2.7/site-packages/conda_build/api.py", line 97, in build
    need_source_download=need_source_download, config=config)
  File "/home/pfernique/.miniconda/lib/python2.7/site-packages/conda_build/build.py", line 1510, in build_tree
    config=config)
  File 